In [ ]:
# default_exp predict

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

# Prediction
> Functions that wrap prediction tasks

All predictions are given as a score between 0 (for grade 1) and 1 (for grade 3).

In [ ]:
# export
from grade_classif.data.utils import load_batches
from grade_classif.models.plmodules import GradesClassifModel
from grade_classif.data.read import get_scan

In [ ]:
# export
def predict_one_scan_one_level(model, fn):
    preds = []
    for x in load_batches(fn, bs=model.bs, device=model.main_device):
        preds.append(model.predict(x).detach().cpu()[:, 1])
    return sum(preds)/len(preds)

Returns - for one zoom level - the prediction for one single scan `fn` (as a `Path` to the corresponding folder) using `model`. Returns the mean result of all patches.

In [ ]:
# export
def predict_one_scan(hparams):
    preds = []
    for level, version, norm_version in zip(hparams.levels, hparams.versions, hparams.norm_versions):
        hparams.level = level
        hparams.norm_version = norm_version
        model = GradesClassifModel(hparams)
        model.load(version)
        path = get_scan(hparams.full_data/f'{hparams.full_data.name}_{level}', hparams.scan, include=['1', '3'])
        preds.append(predict_one_scan_one_level(model, path))
    return sum(preds)/len(preds)

Returns a prediction for one scan over multiple levels. All parameters (such as levels for prediction, models to use or scan to predict) are defined in `hparams`.

In [ ]:
# export
def predict_all(hparams):
    df = pd.read_csv(hparams.data_csv, header='infer')
    preds = []
    scans = []
    for scan in df.loc[df['split']=='valid', 'scan'].values:
        hparams.scan = scan
        scans.append(scan)
        preds.append(predict_one_scan(hparams))
    res = pd.DataFrame({'scan': scans, 'pred': preds})
    return res

Compute predictions for all validation scans. All parameters are defined in `hparams`. This function returns a dataframe that contains the score for each scan.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data_read.ipynb.
Converted 11_data_loaders.ipynb.
Converted 12_data_dataset.ipynb.
Converted 13_data_utils.ipynb.
Converted 14_data_transforms.ipynb.
Converted 20_models_plmodules.ipynb.
Converted 21_models_modules.ipynb.
Converted 22_models_utils.ipynb.
Converted 23_models_hooks.ipynb.
Converted 24_models_metrics.ipynb.
Converted 80_params_defaults.ipynb.
Converted 81_params_parser.ipynb.
Converted 99_index.ipynb.
